In [24]:
# Import some useful libraries
%matplotlib inline
import pandas as pd
import urllib
import xml.etree.ElementTree as ET
import numpy as np
import nltk
import rake
from bs4 import BeautifulSoup as BSoup
%load_ext autoreload
%autoreload 2

# display all pandas columns
pd.set_option('display.max_columns', 100)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# NLP content extraction

First attempt of transcript content extraction
(Tokenization|Parser|Text Segmentation|Word Break rules|Text Analysis)

http://gerardnico.com/wiki/natural_language/parser

### CSV load

In [47]:
df_transcripts = pd.read_csv("../scraping/data/Transcript.csv", nrows=50)
df_transcripts = df_transcripts.loc[df_transcripts['LanguageOfText'] == 'FR']
df_transcripts.head()

,Unnamed: 0,CantonAbbreviation,CantonId,CantonName,CouncilId,CouncilName,DisplaySpeaker,End,Function,ID,IdSession,IdSubject,Language,LanguageOfText,MeetingCouncilAbbreviation,MeetingDate,MeetingVerbalixOid,Modified,ParlGroupAbbreviation,ParlGroupName,PersonNumber,SortOrder,SpeakerFirstName,SpeakerFullName,SpeakerLastName,Start,Text,Type,VoteId
0,0,VD,22.0,Vaud,1.0,Conseil national,True,1999-12-06T14:50:21.663,NaN,1,4601,1,FR,FR,N,19991206,637,2015-11-09T11:48:16.91,C,Groupe démocrate-chrétien,498.0,3,Jacques,Neirynck Jacques,Neirynck,1999-12-06T14:36:50.007,<pd_text><p>La séance d'aujourd'hui s'ouvre so...,1,NaN
2,2,VD,22.0,Vaud,1.0,Conseil national,True,1999-12-06T14:33:39.803,NaN,3,4601,1,FR,FR,N,19991206,637,2015-11-09T11:48:16.937,C,Groupe démocrate-chrétien,498.0,1,Jacques,Neirynck Jacques,Neirynck,1999-12-06T14:32:24.303,<pd_text><p>Madame la Présidente de la Confédé...,1,NaN
11,11,GE,25.0,Genève,1.0,Conseil national,True,1999-12-06T15:23:17.337,NaN,12,4601,2,FR,FR,N,19991206,637,2015-11-09T11:48:16.95,G,Groupe écologiste,497.0,3,Patrice,Mugny Patrice,Mugny,1999-12-06T15:19:52.32,<pd_text><p>Il semble acquis qu'une majorité d...,1,NaN
12,12,VD,22.0,Vaud,1.0,Conseil national,True,1999-12-06T15:26:02.85,NaN,13,4601,2,FR,FR,N,19991206,637,2015-11-09T11:48:16.953,-,Non inscrit,241.0,4,Josef,Zisyadis Josef,Zisyadis,1999-12-06T15:23:17.337,"<pd_text><p>Ainsi donc, la rotation immuable d...",1,NaN
13,13,NE,24.0,Neuchâtel,1.0,Conseil national,True,1999-12-06T15:26:40.757,NaN,14,4601,2,FR,FR,N,19991206,637,2015-11-09T11:48:16.953,L,Groupe libéral,193.0,5,Rémy,Scheurer Rémy,Scheurer,1999-12-06T15:26:02.85,<pd_text><p>Vous venez de recevoir une feuille...,1,NaN


### HTML tag clean up

In [57]:
def clean_html(text_html):
    clean_text = BSoup(text_html, 'lxml').text
    clean_text = clean_text.replace('\n', ' ').replace('\r', '')
    return clean_text

text_html = df_transcripts['Text'][11]
text = clean_html(text_html)
text

"Il semble acquis qu'une majorité du Conseil national élira comme président de notre Conseil M. Hanspeter Seiler, membre de l'UDC. (Applaudissements) Il semble également acquis que le 15 décembre 1999, à moins que l'annonce de la candidature de M. Blocher ne bouscule les choses, un autre UDC, M. Adolf Ogi, deviendra président de la Confédération. Je ne connais pas M. Seiler, et je sais que M. Ogi n'est ni xénophobe ni raciste. Il n'en demeure pas moins que ces deux hommes représentent un parti qui utilise la xénophobie comme arme politique. Le symbole sera puissant et perçu en particulier à l'étranger comme l'écho de la réalité helvétique, alors que nous soutenons une Suisse ouverte au monde et solidaire. Lorsque j'entends des représentants de la droite, et non des moindres, déclarer aujourd'hui qu'il faut aller encore plus dans le sens de l'UDC, je me dis avec une inquiétude profonde que cette élection de deux présidents UDC, si elle se fait, reflétera bel et bien l'état d'esprit poli

### Extract all word, stemmize them, delete stopword, ...
(example with french)

In [92]:
import re
def getWords(text):
    text = text.replace("'", " ")
    list_words = re.compile('\w+').findall(text)
    return list_words

print("==================================")
print("From text to list of word:")
print("==================================")
list_words = getWords(text)
print(list_words)

print("===================================")
print("Stemming")
print("==================================")
stemmer = nltk.stem.snowball.FrenchStemmer()

list_stems = list(map(stemmer.stem, list_words))
print(list_stems)

print("===================================")
print("New text")
print("==================================")
stemmed_text = " ".join(list_stems)
print(stemmed_text)

From text to list of word:
['Il', 'semble', 'acquis', 'qu', 'une', 'majorité', 'du', 'Conseil', 'national', 'élira', 'comme', 'président', 'de', 'notre', 'Conseil', 'M', 'Hanspeter', 'Seiler', 'membre', 'de', 'l', 'UDC', 'Applaudissements', 'Il', 'semble', 'également', 'acquis', 'que', 'le', '15', 'décembre', '1999', 'à', 'moins', 'que', 'l', 'annonce', 'de', 'la', 'candidature', 'de', 'M', 'Blocher', 'ne', 'bouscule', 'les', 'choses', 'un', 'autre', 'UDC', 'M', 'Adolf', 'Ogi', 'deviendra', 'président', 'de', 'la', 'Confédération', 'Je', 'ne', 'connais', 'pas', 'M', 'Seiler', 'et', 'je', 'sais', 'que', 'M', 'Ogi', 'n', 'est', 'ni', 'xénophobe', 'ni', 'raciste', 'Il', 'n', 'en', 'demeure', 'pas', 'moins', 'que', 'ces', 'deux', 'hommes', 'représentent', 'un', 'parti', 'qui', 'utilise', 'la', 'xénophobie', 'comme', 'arme', 'politique', 'Le', 'symbole', 'sera', 'puissant', 'et', 'perçu', 'en', 'particulier', 'à', 'l', 'étranger', 'comme', 'l', 'écho', 'de', 'la', 'réalité', 'helvétique', '

## Rake

`rake_object = rake.Rake("SmartStoplist.txt", 5, 3, 4)`
Now, we have a RAKE object that extracts keywords where:
* Each word has at least 5 characters
* Each phrase has at most 3 words
* Each keyword appears in the text at least 4 times

In [ ]:
rake_object = rake.Rake("FrenchStoplist.txt", 2, 10, 1)
#keywords = rake_object.run(text)
keywords = rake_object.run(stemmed_text)

print("Keywords:")
for k in keywords:
    print(k)

## Maui

**Maui** seems to be a strong improvement of **KEA** and show better scores as far as I could test it. 

(Test done on RAKE-tutorial https://github.com/zelandiya/RAKE-tutorial
following this explanations: https://www.airpair.com/nlp/keyword-extraction-tutorial)

Maui works with supervised and/or unsupervised learning, so we need to create a corpus with Transcripts.

**Advantage:** Maui can work with a controlled vocabulary to be more consitant! Idea of pipeline:
* Create a Transcript corpus (with clean text but without any other processing (no stemming, ...)
* Train Maui and then test the obtained result agains my classification
* Define a bench of topic in which classified the texts (and expand them with synonyme from, for example, Wordnet)
* Train Maui again with this controlled vocabulary
* Pray to obtain some results... 

In [ ]:
# Create a .txt corpus (in french for the moment)
df_transcripts = pd.read_csv("../scraping/data/Transcript.csv", nrows=50)
df_transcripts = df_transcripts.loc[df_transcripts['LanguageOfText'] == 'FR']
df_transcripts.head()